In [1]:
# Import Necessary Libraries
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import multivariate_normal, bernoulli, beta, norm
from scipy.stats import ks_2samp
from scipy.special import expit as logistic_sigmoid
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.metrics import brier_score_loss
from sklearn.model_selection import cross_val_score
from sklearn.metrics import balanced_accuracy_score, brier_score_loss, accuracy_score, roc_auc_score, roc_curve, auc
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
import pickle

np.random.seed(42)

In [2]:
df = pd.read_csv('../regression/test_dataset.csv')
model = pickle.load(open('../regression/regression_model_MM.sav', 'rb'))

In [3]:
def custom_predict(input_data):
    # Preprocess input_data if necessary
    input_data = np.array(input_data)
    predictions = model.predict(input_data)
    return predictions.tolist()

In [14]:
# Convert dataframe to list of dictionaries (required format for WIT)
examples = df.to_dict(orient='records')

# Configure the What-If Tool
config_builder = WitConfigBuilder(examples).set_custom_predict_fn(custom_predict)

# Launch the What-If Tool
WitWidget(config_builder)

WitWidget(config={'model_type': 'classification', 'label_vocab': [], 'uses_json_input': True, 'inference_addre…

In [6]:
import subprocess

# Enable the widgetsnbextension
subprocess.run(["jupyter", "nbextension", "enable", "--py", "widgetsnbextension", "--sys-prefix"])

# Install and enable the widgetsnbextension
subprocess.run(["jupyter", "nbextension", "install", "--py", "widgetsnbextension", "--sys-prefix"])

# Enable the witwidget extension
subprocess.run(["jupyter", "nbextension", "enable", "--py", "witwidget", "--sys-prefix"])


CompletedProcess(args=['jupyter', 'nbextension', 'enable', '--py', 'witwidget', '--sys-prefix'], returncode=1)